# Analyzing and Transforming Abstract Syntax

You manage a large Python codebase that interacts with another vendor's library. A new version of this library is not backwards compatible: it has switched from lists to iterators throughout its API. This is a problem because your codebase uses [slice notation](https://docs.python.org/3/library/functions.html?highlight=slice#slice) on list values throughout, which must now be replaced with calls to [`islice`](https://docs.python.org/3/library/itertools.html#itertools.islice). You are tasked with quantifying the extent to which the codebase under your management must be modified, and with designing and/or implementing a process for performing the updates across this codebase. How can you avoid having to manually identify and rewrite every occurrence of slice notation? Is there a way both to measure the extent of the problem and to perform the necessary code transformations in an automated way?

More generally, the ability to work directly with the abstract syntax data structures of a language can be useful in at least two kinds of scenarios.
* It may be necessary to *analyze* code (possibly a very large amount of code) in an automated way to obtain some useful information (*e.g.*, whether patterns that may represent vulnerabilities or errors exist, where and how often a language feature is used, the portion of the code for which tests and documentation are present, and so on).
* It may be necessary to *transform* code (again, possibly a very large amount of code) in some way (*e.g.*, to update its compatibility with a newer version of an API, to transpile it to another language, and so on).

Python's built-in libraries include powerful modules that make it possible to retrieve and operate over parse trees or *abstract syntax trees* (ASTs) of modules, classes, and functions/methods. This article provides an overview of the Python syntax definition, how to build and operate on abstract syntax trees, and some use cases. With this information in hand, you should be ready to start writing Python code that examines and transforms other Python code.

## Abstract Syntax Grammar and Data Structure

For any given programming language, the *concrete syntax* corresponds to the actual strings of characters that make up a program, and the *abstract syntax* corresponds to the data structure instances that a parser for that language generates from a string of characters (and which may subsequently be processed by an interpreter or a compiler). An instance of the abstract syntax data structure is sometimes called a *parse tree*, an *abstract syntax tree*, or an *AST*.

The definitions of abstract syntax grammar for Python is presented in the [documentation for the `ast` module](https://docs.python.org/3/library/ast.html#abstract-grammar). The `ast` module itself includes an implementation of the data structure corresponding to the grammar definition, as well as associated functionalities (such as featues that enable parsing a concrete syntax string into an AST instance and performing transformations on an AST instance).

In [1]:
import ast

# Programmatically build an AST for the
# Python concrete syntax expression `-5`.
node =\
  ast.UnaryOp(
    ast.USub(),
    ast.Num(5, lineno=0, col_offset=0),
    lineno=0,
    col_offset=0
  )

ast.dump(node)

'UnaryOp(op=USub(), operand=Constant(value=5))'

You can use the [`inspect`](https://docs.python.org/3/library/inspect.html) module to retrieve the concrete syntax of the body of a Python function definition.

In [1]:
def f(x):
    return x + 2

import inspect
inspect.getsource(f)

'def f(x):\n    return x + 2\n'

You can then parse the concrete syntax using [`ast.parse()`](https://docs.python.org/3/library/ast.html#ast.parse) to generate an instance of the abstract syntax data structure.

In [1]:
t = ast.parse(inspect.getsource(f))
ast.dump(t)

"Module(body=[FunctionDef(name='f', args=arguments(posonlyargs=[], args=[arg(arg='x', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Constant(value=2, kind=None)))], decorator_list=[], returns=None, type_comment=None)], type_ignores=[])"

How can this data structure be inspected and traversed using the `ast` API? Every instance of the data structure is also a node in the tree that represents the root of the module, statement, or expression that was parsed to build it. For example, you can retrieve the body of the function definition above in the following way.

In [1]:
print(t._fields)
print(t.body)
print(type(t.body[0]) is ast.FunctionDef)
func_body = t.body[0]

('body', 'type_ignores')
True


You can retrieve the single statement in the body of the function.in the following way.

In [1]:
print(func_body._fields)
print(func_body.body)
print(type(func_body.body[0]) is ast.Return)
stmt_return = func_body.body[0]

('name', 'args', 'body', 'decorator_list', 'returns', 'type_comment')
True


Going deeper, you can retrieve the expression inside the `return` statement in the following way.

In [1]:
print(stmt_return._fields)
print(stmt_return.value)
print(type(stmt_return.value) is ast.BinOp)
expr_add = stmt_return.value

('value',)
True


Finally, you can extract the expression operator and its individual arguments.

In [1]:
print(expr_add.left)
print(expr_add.left.id)
print(expr_add.op)
print(expr_add.right)
print(expr_add.right.n)

x
2


## Analyzing an Abstract Syntax Tree

The `ast` API allows you to programmatically write you own functions for operating on Python ASTs. The example below collects all the variables in an AST (as long as the AST consists of the specific kinds of nodes that are handled by the various `if` branches in the body of the function).

In [1]:
def collect_variables(a):
    if type(a) is ast.Module:
        return [v for s in a.body for v in get_variables(s)]
    
    elif type(a) is ast.FunctionDef:
        vs = [v for s in a.body for v in get_variables(s)]
        return [a.name] + vs

    elif type(a) is ast.Assign:
        vs = [v for s in a.targets for v in get_variables(s)]
        return vs + get_variables(a.value)
    
    elif type(a) is ast.Return:
        return get_variables(a.value)

    elif type(a) is ast.Name:
        return [a.id]

    elif type(a) is ast.BinOp:
        return get_variables(a.left) + get_variables(a.right)
    
    else:
        print(type(a))
        return []

In [1]:
def f(x, y):
    a = b
    u = 1 + 2
    z = x + y
    return z

a = ast.parse(inspect.getsource(f))
collect_variables(a)

<class '_ast.Constant'>
<class '_ast.Constant'>


['f', 'a', 'b', 'u', 'z', 'x', 'y', 'z']

You can instead use the `ast.walk()` function to traverse a tree if you are not worried about its internal structure but wish to see and process all of its nodes in some way (usually, to produce some kind of less granular aggregate analysis). The example below performs the same task as the function `collect_variables` above.

In [1]:
vs = []
for node in ast.walk(a):
    if isinstance(node, ast.FunctionDef):
        vs.append(node.name)
    if isinstance(node, ast.Name):
        vs.append(node.id)
vs

['f', 'a', 'b', 'u', 'z', 'z', 'x', 'y']

## Transforming an Abstract Syntax Tree

You can use the `ast.NodeVisitor` and `ast.NodeTransformer` base classes to create your own algorithms that traverse or modify ASTs. This allows your implementation to have a more structured approach than one that uses `ast.walk`, while also allowing you to avoid explicitly handling every possible kind of AST node in your implementation.

In [1]:
class VariableLister(ast.NodeVisitor):
    def visit_Name(self, node):
        print(node.id)
        self.generic_visit(node)
        
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node) # Needed since we are overriding method.

In [1]:
def f(x, y):
    a = b
    u = 1 + 2
    z = x + y
    return z

a = ast.parse(inspect.getsource(f))
VariableLister().visit(a)

f
a
b
u
z
x
y
z


In [1]:
class VariableChanger(ast.NodeTransformer):

    def visit_Name(self, node):
        a =\
          ast.copy_location(
            ast.Subscript(
              value=ast.Name(id='data', ctx=ast.Load()),
              slice=ast.Index(value=ast.Str(s=node.id)),
              ctx=node.ctx
            ),
            node
          )
        return a

In [1]:
def f():
    return x + y

f_new_ast = VariableChanger().visit(ast.parse(inspect.getsource(f)))
ast.dump(f_new_ast)

"Module(body=[FunctionDef(name='f', args=arguments(posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Subscript(value=Name(id='data', ctx=Load()), slice=Index(value=Constant(value='x')), ctx=Load()), op=Add(), right=Subscript(value=Name(id='data', ctx=Load()), slice=Index(value=Constant(value='y')), ctx=Load())))], decorator_list=[], returns=None, type_comment=None)], type_ignores=[])"

You can use the [`astor`](https://astor.readthedocs.io) module to convert an AST back into concrete syntax. This is a straightforward way to write transformed code back to a file.

In [1]:
import astor
print(astor.to_source(f_new_ast))

def f():
    return data['x'] + data['y']

